In [1]:
from inspect import Signature, Parameter

# Make a signature for a func(x, y=42, *, z=None)
parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
    Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
    Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
sig = Signature(parms)
print(sig)


(x, y=42, *, z=None)


In [2]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name,value)

In [3]:
# Try various examples
func(1, 2, z=3)

x 1
y 2
z 3


In [5]:
func(1)

x 1


In [6]:
func(1, z=3)

x 1
z 3


In [7]:
func(y=2, x=1)

x 1
y 2


In [8]:
func(1, 2, 3, 4)

TypeError: too many positional arguments

In [9]:
func(y=2)

TypeError: missing a required argument: 'x'

In [10]:
func(1, y=2, x=3)

TypeError: multiple values for argument 'x'

In [11]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
        for name in names]
    return Signature(parms)
class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)
# Example use
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')
class Point(Structure):
    __signature__ = make_sig('x', 'y')

In [12]:
import inspect
print(inspect.signature(Stock))

(name, shares, price)


In [13]:
s1 = Stock('ACME', 100, 490.1)

In [14]:
s2 = Stock('ACME', 100)

TypeError: missing a required argument: 'price'

In [15]:
s3 = Stock('ACME', 100, 490.1, shares=50)

TypeError: multiple values for argument 'shares'

In [16]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
        for name in names]
    return Signature(parms)
class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields',[]))
        return super().__new__(cls, clsname, bases, clsdict)
class Structure(metaclass=StructureMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)
# Example
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    
class Point(Structure):
    _fields = ['x', 'y']

In [17]:
import inspect
print(inspect.signature(Stock))

(name, shares, price)


In [18]:
print(inspect.signature(Point))

(x, y)
